In [1]:
#!python -m spacy download en_core_web_sm

In [2]:
import numpy as np
import pandas as pd

import re
import string
import spacy
import spacy_cleaner




from sklearn.feature_extraction.text import TfidfVectorizer          
from sklearn.metrics.pairwise import cosine_similarity               

import gensim                                       
import gensim.downloader
from gensim.models import word2vec,KeyedVectors


In [3]:
df = pd.read_csv('/home/user/spacy/archive/sample-data.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           500 non-null    int64 
 1   description  500 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [4]:
df['description'][0]

'Active classic boxers - There\'s a reason why our boxers are a cult favorite - they keep their cool, especially in sticky situations. The quick-drying, lightweight underwear takes up minimal space in a travel pack. An exposed, brushed waistband offers next-to-skin softness, five-panel construction with a traditional boxer back for a classic fit, and a functional fly. Made of 3.7-oz 100% recycled polyester with moisture-wicking performance. Inseam (size M) is 4 1/2". Recyclable through the Common Threads Recycling Program.<br><br><b>Details:</b><ul> <li>"Silky Capilene 1 fabric is ultralight, breathable and quick-to-dry"</li> <li>"Exposed, brushed elastic waistband for comfort"</li> <li>5-panel construction with traditional boxer back</li> <li>"Inseam (size M) is 4 1/2"""</li></ul><br><br><b>Fabric: </b>3.7-oz 100% all-recycled polyester with Gladiodor natural odor control for the garment. Recyclable through the Common Threads Recycling Program<br><br><b>Weight: </b>99 g (3.5 oz)<br><b

In [5]:
model = spacy.load("en_core_web_sm")
model.Defaults.stop_words.add('oz')
cleaner = spacy_cleaner.Cleaner( 
                                model=model,
                                remove_numbers = True,
                                remove_punctuation = True,
                                remove_stopwords = True,
                                remove_email = True,
                                remove_url = True 
                                                    )

In [6]:
def clear_text(text):
    html=re.compile(r'<.*?>')
    out = html.sub(r' ',text)
    return cleaner.clean([out])[0]


In [7]:
series_ = df['description'].apply(lambda x: clear_text(x))

Cleaning Progress: 100%|██████████████████████████| 1/1 [00:00<00:00, 26.28it/s]


Cleaning Progress: 100%|██████████████████████████| 1/1 [00:00<00:00, 16.64it/s]


Cleaning Progress: 100%|██████████████████████████| 1/1 [00:00<00:00, 17.18it/s]


Cleaning Progress: 100%|██████████████████████████| 1/1 [00:00<00:00, 28.53it/s]


Cleaning Progress: 100%|██████████████████████████| 1/1 [00:00<00:00, 28.53it/s]


In [8]:
series_[0]

'active classic boxers reason boxers cult favorite cool especially sticky situations quick drying lightweight underwear takes minimal space travel pack exposed brushed waistband offers skin softness panel construction traditional boxer classic fit functional fly recycled polyester moisture wicking performance inseam size m recyclable common threads recycling program details silky capilene fabric ultralight breathable quick dry exposed brushed elastic waistband comfort panel construction traditional boxer inseam size m fabric recycled polyester gladiodor natural odor control garment recyclable common threads recycling program weight g mexico'

In [9]:
df['description'] = series_

In [10]:
tfidf_vectorizer = TfidfVectorizer(binary=True, max_features=25000)
tfidf_embedings = tfidf_vectorizer.fit_transform(df['description'])
tfidf_embedings.shape


(500, 4326)

In [11]:
cos_matrix = cosine_similarity(tfidf_embedings)
pair_indexes = np.transpose(np.nonzero(cos_matrix > 0.8))

for x in pair_indexes:
    if x[0] != x[1]:
        print(df['description'][x])

3      alpine guide pants skin climb ice switch rock ...
158    alpine guide pants skin climb ice switch rock ...
Name: description, dtype: object
4      alpine wind jkt high ridges steep ice alpine j...
307    alpine wind jkt high ridges steep ice alpine j...
Name: description, dtype: object
7      print banded betina btm fullest coverage botto...
219    solid betina btm fullest coverage bottoms beti...
Name: description, dtype: object
26    compound cargo pants long ultimate pants built...
27    compound cargo pants reg ultimate pants built ...
Name: description, dtype: object
26     compound cargo pants long ultimate pants built...
451    compound cargo pants short ultimate pants buil...
Name: description, dtype: object
27    compound cargo pants reg ultimate pants built ...
26    compound cargo pants long ultimate pants built...
Name: description, dtype: object
27     compound cargo pants reg ultimate pants built ...
451    compound cargo pants short ultimate pants buil...
Name: de

##  W2V

In [12]:
word_vectors = gensim.downloader.load("word2vec-google-news-300")


In [13]:
docs_vectors = pd.DataFrame()

for doc in df['description']:
    temp = pd.DataFrame()
    for word in doc.split(' '):   
        try:
            word_vec = word_vectors[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
        
    doc_vector = temp.mean()
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)

docs_vectors.shape



/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning

/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: Futur

/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: Futur

/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: Futur

/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: Futur

...

/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: Futur

/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: Futur

/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: Futur

/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series(word_vec), ignore_index = True)
/tmp/ipykernel_661753/1707741832.py:8: FutureWarning

(500, 300)

In [14]:
docs_vectors.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
397,-0.052001,0.051070,-0.073582,0.034416,-0.081988,-0.002850,0.009911,-0.107146,0.041589,0.113869,...,-0.078194,0.008950,-0.061232,0.006480,0.018725,-0.021165,0.059143,-0.009810,-0.020751,0.017292
87,-0.008827,0.059525,-0.052329,0.046337,-0.073995,0.033789,-0.001593,-0.106623,0.046724,0.148521,...,-0.085156,0.049857,-0.059245,0.048127,-0.014872,-0.011612,0.066372,0.004058,0.045643,0.005599
361,-0.043193,0.067403,-0.011990,0.050640,-0.068566,-0.016846,0.049756,-0.100759,0.060486,0.096421,...,-0.027258,0.025361,-0.034455,0.034543,-0.002525,-0.009192,0.041307,-0.038576,0.023528,0.043482
168,-0.048776,0.062907,-0.031354,0.109744,-0.074240,0.018463,0.082588,-0.095604,0.057234,0.110847,...,-0.077792,-0.008611,-0.034255,-0.035015,0.025851,-0.030042,0.102517,-0.044629,0.022406,0.021511
108,-0.049732,0.065055,-0.033625,0.161819,-0.069332,0.015039,0.051203,-0.171285,0.068867,0.097653,...,-0.054752,0.009500,-0.043922,0.020456,0.037071,-0.069668,0.110229,-0.030840,0.075605,0.041591


In [15]:
cos_matrix_w2v = np.around(cosine_similarity(docs_vectors), 5)
cos_matrix_w2v

array([[1.     , 0.92909, 0.90296, ..., 0.88944, 0.88636, 0.89687],
       [0.92909, 1.     , 0.95206, ..., 0.88214, 0.89081, 0.8968 ],
       [0.90296, 0.95206, 1.     , ..., 0.86952, 0.85091, 0.86196],
       ...,
       [0.88944, 0.88214, 0.86952, ..., 1.     , 0.86365, 0.85841],
       [0.88636, 0.89081, 0.85091, ..., 0.86365, 1.     , 0.94982],
       [0.89687, 0.8968 , 0.86196, ..., 0.85841, 0.94982, 1.     ]],
      dtype=float32)

In [16]:
pair_indexes_w2v = np.transpose(np.nonzero(cos_matrix_w2v > 0.985))

for x in pair_indexes_w2v:
    if x[0] != x[1]:
        print(df['description'][x])

3      alpine guide pants skin climb ice switch rock ...
158    alpine guide pants skin climb ice switch rock ...
Name: description, dtype: object
4      alpine wind jkt high ridges steep ice alpine j...
307    alpine wind jkt high ridges steep ice alpine j...
Name: description, dtype: object
14    borderless shorts-11 forward cringe borderless...
15    borderless shorts-7 summertime gifts chance fr...
Name: description, dtype: object
15    borderless shorts-7 summertime gifts chance fr...
14    borderless shorts-11 forward cringe borderless...
Name: description, dtype: object
19     cap graphic t shirt tee lightest weight capile...
339    cap graphic t shirt elegant union graceful des...
Name: description, dtype: object
19     cap graphic t shirt tee lightest weight capile...
487    cap graphic crew choice rest days include ulti...
Name: description, dtype: object
21     cap t shirt revels sweat jamborees hill climbs...
359    cap t shirt like sweet breeze long runs hot da...
Name: de